In [41]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os
from bs4 import BeautifulSoup
import re
from datetime import datetime



In [61]:
def prepros(html) :
    d=json.loads(html)
    
    soup = BeautifulSoup(d['result_list_box_html'],'lxml')
    
    # this selects the main part of the html
    tabel = soup.find('div', attrs={'class':'results component--default'})
    
    # Jobindex contain both own postings and postings from other
    # the two are treated differently in the data
    # Jobindex' imported postings
    others = re.compile('r[\d]+')
    tabel1 = tabel.findAll('div', attrs={'data-beacon-tid':others})
#    print(tabel1)

    # Jobindex' own postings
    own = re.compile('h[\d]+')
    tabel2 = tabel.findAll('div', attrs={'data-beacon-tid':own})
    #print(tabel2)
    
    return tabel, tabel1, tabel2

def job_descr_own(tabel2) : # den er ikke færdig: tag <ul> og saml elementer, der hører sammen
    desc_own = []

    # tag ul skal med 
    for l in tabel2 :
        g = l.findAll('p')
        for m in g :
            desc_own.append(m.text)
    return desc_own

def job_descr_oth(tabel1) : # færdig og virker
    desc_oth = []
    p=re.compile('"')
    lineshift = re.compile('\n')
    besk = []
    for t_ in tabel1 :
        besk.append(lineshift.sub("", t_.text))
    for t in besk :
        s = t.split(sep='    ')
        if len(s) == 1 :
            desc_oth.append("")
        else :
            desc_oth.append(t.split(sep='    ')[1])
    return desc_oth
    
def job_title_oth (tabel) : # færdig og virker
    # udled jobs andre
    jobs_oth = []
    j = tabel.findAll('strong') 
    for l in j :
        jobs_oth.append(l.text)
    return jobs_oth


def firm_place(tabel) : # tjek den igen, der kommer lidt for mange elementer ud
    firm=[]
    city=[]
    for tag in tabel :
        firm_city=tag.findAll('b')
        if len(firm_city) == 0 :
            firm.append("Ukendt")
            city.append("Uoplyst")
        elif len(firm_city) == 1 :
            firm.append(firm_city[0].text)
            city.append(" ")
        else :
            firm.append(firm_city[0].text)
            city.append(firm_city[1].text)
    return firm, city


def dates(tabel) : # finds only dates for others; needs also dates for own
    indented_d=[]
    for tag in tabel :
        dato_site=tag.findAll('time')

        monthval={'januar':'01','februar':'02', 'marts':'03', 'april':'04', 'maj':'05', 'juni':'06', 'juli':'07','august':'08','september':'09','oktober':'10','november':'11','december':'12'}

        for i in dato_site:
            t = i.text.split()
            try :
                datotal=(t[0][:-1]+ monthval.get(t[1])+ t[2])
                indented_d.append(datetime.datetime(t[2], monthval.get(t[1]), t[0][:-1]))
#                indented_d.append(time.mktime(datetime.strptime(str(datotal), '%d%m%Y').timetuple()))
            except :
                indented_d.append(time.time()) # vi bør videreføre sidst kendte værdi også på tværs af sider

    return indented_d



#hyper = []
#x1 = tabel.findAll('a', attrs={'href':True}) 
#for x in x1 :
#    hyper.append(x.)
#    
#print(x1)



In [20]:
# select small subsample
import random, time

ca_samplesize = 10
base_path = r"C:\Notebooks/jobindex.txt"
sample_path = r"C:\Notebooks\jobindex_sample.txt"
f = open(base_path,'r')
s = open(sample_path, 'w')
linienr = 0
t0 = time.time()
for line in f :
    ran = random.uniform(0, 1) 
    if ran < ca_samplesize/17000 :
        linienr += 1
        s.write(line)
f.close()
t1 = time.time()
print("Start:", t0, ". Slut:", t1, ". Linier hentet:", linienr)
print("Kørseltid:", round(t1-t0,2), "sekunder")




Start: 1535305816.4684303 . Slut: 1535305829.1954093 . Linier hentet: 13
Kørseltid: 12.73 sekunder


In [63]:

def process_data() :
    t0 = time.time()
    base_path = r"C:\Notebooks\jobindex_sample.txt" # file with scraped jobindex data

    f = open(base_path, mode='r') # open the file for reading # jobindex2 skal læses med , encoding='utf8'
    count = 0
    # loop through the file one line at a time

    output_oth = pd.DataFrame(columns=['date', 'job_title', 'job_describ', 'city', 'company'])
#    output_own = pd.DataFrame()
    
   
    for html in f :
        count += 1
#        print(count, len(html))
        if len(html) == 1 :
            continue
        # here the various processing functions will be called
        tabel, tabel1, tabel2= prepros(html)
#        print(len(tabel))

        desc_own = job_descr_own(tabel2)
#        print("desc_own", len(desc_own), desc_own)

        desc_oth = job_descr_oth(tabel1)
#        print('desc_oth', len(desc_oth), desc_oth)
        
        jobs_oth = job_title_oth(tabel)
#        print('jobs_oth', len(jobs_oth), jobs_oth)
        
        city_oth, firm_oth = firm_place(tabel1)
        
        post_dates_oth = dates(tabel1)
#        print("dates", len(post_dates_oth))

        post_dates_own = dates(tabel1)
#        print("dates", len(post_dates_own), post_dates_own)

    # der skal laves en zip-funktion, så de kan blive sat rigtig sammen
#    return desc_own, desc_oth, jobs_oth, firm, city, post_dates
        tmp = pd.DataFrame(list(zip(post_dates_oth, jobs_oth, desc_oth, firm_oth, city_oth)), 
                           columns=['date', 'job_title', 'job_describ', 'city', 'company'])
        output_oth = pd.concat([output_oth, tmp], axis=0, join='inner', ignore_index=True)
#        zip_own = pd.DataFrame(list(zip(post_dates_own, desc_own)))
#        output_own.add(zip_own)
    f.close()
    t1 = time.time()
    print("Procestid er", round(t1-t0, 2), "sekunder")
    #print(output_oth)
    return output_oth#, output_own

output_oth = process_data()



Procestid er 1.41 sekunder


In [70]:
# gem data

print(output_oth.shape)

output_oth.to_pickle('others_sample.pkl')

#own = pd.DataFrame(output_own)
#own.to_pickle('own.pkl')


#print(len(desc_own))
#print(len(desc_oth))
#print(len(jobs_oth))
#print(len(firm))
#print(len(city))
#print(len(post_dates))

(220, 5)


In [65]:

# læs data 
ny = pd.read_pickle('others_sample.pkl')

print("Antallet af records", len(ny))
print("\n")
print(ny.head(1), "\n", ny.tail(2))


Antallet af records 220


           date           job_title  \
0  1.535350e+09  Servicemedarbejder   

                                         job_describ      city        company  
0  Opgaverne kan være lettere vareopfyldning opry...  Hillerød  Netto Danmark   
              date                                  job_title  \
218  1.535350e+09                        Sejlmager søges ...   
219  1.535350e+09  Sundhedstjenesten søger sundhedsplejerske   

                                           job_describ         city  \
218  ... eller tillært, der har erfaring med båd ka...  Skanderborg   
219  Sundhedstjenesten søger per 1. oktober 2008 en...  Skanderborg   

                    company  
218  Skanderborg Sejlmageri  
219     Skanderborg kommune  


In [71]:
#import counter

ny.head()

,date,job_title,job_describ,city,company
0,1.535350e+09,Servicemedarbejder,Opgaverne kan være lettere vareopfyldning opry...,Hillerød,Netto Danmark
1,1.535350e+09,Familiekonsulent til Poppelgården Familiecenter,Hvidovre Kommunes familiecenter Poppelgården s...,Hvidovre,Poppelgården Familiecenter
2,1.535350e+09,Client Service Director,For at blive oprettet som virksomhedsbruger - ...,København,Resources
3,1.535350e+09,Jobkonsulent primært til flygtninge-/indvandre...,For at blive oprettet som virksomhedsbruger - ...,Ringkøbing,Ringkøbing-Skjern Kommune
4,1.535350e+09,Pædagog til børn med særlige behov,For at blive oprettet som virksomhedsbruger - ...,Ringkøbing,Ringkøbing-Skjern Kommune
